In [ ]:
%pip install mysql-connector-python

In [ ]:
import mysql.connector 

In [ ]:
from mysql.connector.pooling import MySQLConnectionPool

In [ ]:
from mysql.connector import  pooling, Error

In [ ]:
try:
    connection_config_dict = {
        'host': 'localhost',
        'user': 'root',
        'database': 'little_lemon_db',
        'autocommit': False
    }

    connection = mysql.connector.connect(**connection_config_dict)

    # Create a connection pool
    pool = pooling.MySQLConnectionPool(pool_name="little_lemon_pool",
                                       pool_size=10,
                                       **connection_config_dict)

    print("The connection pool is created with the name:", pool.pool_name, "and the pool size is:", pool.pool_size)

    if connection.is_connected():
        cursor = connection.cursor(buffered=True)

        # Use the database
        cursor.execute("USE little_lemon_db")
        print("Using little_lemon_db")

        customer_detail = """
            SELECT 
                Bookings.BookingID, 
                CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS Customer_Name,
                SUM(Orders.BillAmount) AS Total_BillAmount
            FROM 
                Bookings
            INNER JOIN 
                Orders ON Bookings.BookingID = Orders.BookingID
            GROUP BY 
                Bookings.BookingID, 
                Customer_Name
            HAVING 
                Total_BillAmount > 60
            ORDER BY 
                SUM(Orders.Quantity) ASC, 
                Total_BillAmount ASC;
        """
        cursor.execute(customer_detail)
        result = cursor.fetchall()
        connection.commit()

        for row in result:
            print(row)



except Error as e:
    print("Error while connecting to MySQL", e)
    connection.rollback()

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'connection' in locals() and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")
